# 大语言模型与pandas数据分析应用

langchain下的pandas代理

https://api.python.langchain.com/en/latest/agents/langchain_experimental.agents.agent_toolkits.pandas.base.create_pandas_dataframe_agent.html

pandas-ai

https://pandas-ai.com/

pandas数据分析与sql数据分析比较

In [1]:
from langchain.utilities import PythonREPL
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.output_parsers import JsonOutputParser
from dashscope import Generation
import dashscope
from langchain.tools.render import render_text_description
from operator import itemgetter
from langchain_core.tools import tool
import pandas as pd
import io
 
dashscope.api_key = ""

## 基于LLM的pandas数据分析代理实现

pythonrepl本质上就是python自带的执行函数 exec(source, globals=None, locals=None, /)

https://blog.csdn.net/huanghong6956/article/details/115454956

langchain封装的执行函数

https://api.python.langchain.com/en/latest/utilities/langchain_experimental.utilities.python.PythonREPL.html

* globals传参



In [3]:
r1=pd.read_csv(r"F:\langchain\15\t_alibaba_data3.txt",names=["user_id","brand_id","behavior_type","date"],sep="\t")

In [9]:
print(df_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182880 entries, 0 to 182879
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        182880 non-null  int64 
 1   brand_id       182880 non-null  int64 
 2   behavior_type  182880 non-null  int64 
 3   date           182880 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.6+ MB



In [7]:
buf = io.StringIO()
r1.info(buf=buf)
df_info = buf.getvalue()

In [13]:
[convert_to_openai_tool(coderunner)]

[{'type': 'function',
  'function': {'name': 'coderunner',
   'description': 'coderunner(code: str) -> dict - python代码执行器',
   'parameters': {'type': 'object',
    'properties': {'code': {'type': 'string'}},
    'required': ['code']}}}]

In [12]:
p=PythonREPL()
p.run("a=9\nprint(a)")

'9\n'

In [14]:
#p

In [15]:
parser = JsonOutputParser()

@tool
def coderunner(code: str) -> dict:
    """python代码执行器"""
    run_code=PythonREPL(_globals={"df":r1})
    res=run_code.run(code)
    if res=='':
        return run_code
    else:
        return res

def get_response_t(messages):
    response = Generation.call(
        model='qwen-max',
        messages=messages,
        tools=[convert_to_openai_tool(coderunner)],
        result_format='message', # 将输出设置为message形式
    )
    return response

def prompt_data(content):
    system_prompt_t=f"""已知代码中数据信息如下：
{df_info}
此数据已经赋值给全局变量df，在已知df的基础上基于pandas编写代码然后调用工具coderunner完成任务,注意统计结果的变量名必须是result_data
"""
    prompt=[{"role":"system","content":system_prompt_t},\
            {"role":"user","content":"总共有多少行记录"},\
            {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'coderunner', 'arguments': '{"code": "import pandas as pd\nresult_data=len(df)"}'}}]}]
    prompt.append({"role":"user","content":content})
    return prompt


In [16]:
p_y=prompt_data("获取每个用户的记录数，并输出前五个数据")
res=get_response_t(p_y)

In [28]:
code=parser.parse(res.output.choices[0].message['tool_calls'][0]['function']['arguments'])["code"]
print(code)

import pandas as pd
user_record_counts = df['user_id'].value_counts()
result_data = user_record_counts.head(5)


In [29]:
run_res=coderunner.invoke(code)

In [33]:
run_res.dict()['locals']['result_data']

user_id
2940000     2407
4184250     1473
3031750     1343
6874250     1275
12215000    1258
Name: count, dtype: int64

In [34]:
p_y=prompt_data("获取用户2940000每天的记录数")
res=get_response_t(p_y)
code=parser.parse(res.output.choices[0].message['tool_calls'][0]['function']['arguments'])["code"]
run_res=coderunner.invoke(code)
run_res.dict()['locals']['result_data']

date
04/16    59
04/17    13
04/18    13
04/19    35
04/20    41
         ..
08/06    27
08/08    50
08/09    16
08/11    24
08/12    17
Name: user_id, Length: 91, dtype: int64

### classwork1

1, 利用上面的函数，完成数据统计问答

2，在streamlit上实现数据统计问答应用

## 结合LLM的pyecharts可视化

pyecharts

https://gallery.pyecharts.org/#/


pyecharts图表在streamlit显示方法

https://github.com/andfanilo/streamlit-echarts

https://cloud.tencent.com/developer/article/2324673

* 引号的问题

In [106]:
a1=[2,3,4]
type(a1)

list

In [35]:
run_res.dict()['locals']['result_data']

date
04/16    59
04/17    13
04/18    13
04/19    35
04/20    41
         ..
08/06    27
08/08    50
08/09    16
08/11    24
08/12    17
Name: user_id, Length: 91, dtype: int64

In [36]:
data_0=run_res.dict()["locals"]["result_data"]
buf = io.StringIO()
data_0.info(buf=buf)
data_0_info = buf.getvalue()

In [38]:
print(data_0_info)

<class 'pandas.core.series.Series'>
Index: 91 entries, 04/16 to 08/12
Series name: user_id
Non-Null Count  Dtype
--------------  -----
91 non-null     int64
dtypes: int64(1)
memory usage: 3.5+ KB



In [40]:
data_0

date
04/16    59
04/17    13
04/18    13
04/19    35
04/20    41
         ..
08/06    27
08/08    50
08/09    16
08/11    24
08/12    17
Name: user_id, Length: 91, dtype: int64

In [41]:
@tool
def visrunner(code: str) -> dict:
    """python代码执行器"""
    run_code=PythonREPL(_globals={"res_data":data_0})
    res=run_code.run(code)
    if res=='':
        return run_code
    else:
        return res

In [49]:
parser = JsonOutputParser()

def get_response_t(messages):
    response = Generation.call(
        model='qwen-max',
        messages=messages,
        tools=[convert_to_openai_tool(visrunner)],
        result_format='message', # 将输出设置为message形式
    )
    return response

def prompt_vis(content):
    system_prompt_t=f"""已知代码中数据信息如下：
{data_0_info}
此数据已经赋值给全局变量res_data，在已知res_data的基础上基于pyecharts编写代码然后务必调用工具visrunner完成任务,注意图形的变量名必须是res_vis
"""
    prompt=[{"role":"system","content":system_prompt_t},\
            {"role":"user","content":"请基于此数据做柱图"},\
            {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'visrunner', 'arguments': '{"code": "from pyecharts import options as opts\nfrom pyecharts.charts import Bar\nres_vis = (Bar().add_xaxis(res_data.index.to_list()).add_yaxis("商家A", res_data.to_list(), stack="stack1").set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(title_opts=opts.TitleOpts(title="Bar-堆叠数据（部分）")))\nres_vis.render()"}'}}]}]
    prompt.append({"role":"user","content":content})
    return prompt


In [44]:
p_y=prompt_vis("请将数据可视化")
res=get_response_t(p_y)

In [54]:
code=res.output.choices[0].message['tool_calls'][0]['function']['arguments'].split('{"code": "')[1].split('"\n}')[0]

In [55]:
result_vis=visrunner.invoke(code)

In [61]:
result_vis.dict()['locals']['res_vis'].render()

'F:\\langchain\\15\\render.html'

In [62]:
result_vis.dict()['locals']['res_vis']

In [ ]:
st_pyecharts(result_vis.dict()['locals']['res_vis'])

In [89]:
res

GenerationResponse(status_code=<HTTPStatus.OK: 200>, request_id='1fc39d6f-59fb-986a-bda9-aa788ed7f61c', code='', message='', output=GenerationOutput(text=None, choices=[Choice(finish_reason='tool_calls', message=Message({'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'visrunner', 'arguments': '{"code": "from pyecharts import options as opts\nfrom pyecharts.charts import Bar\nres_vis = (\n    Bar()\n    .add_xaxis(res_data.index.to_list())\n    .add_yaxis("Count", res_data.tolist(), category_gap="60%")\n    .set_global_opts(title_opts=opts.TitleOpts(title="Counts Bar Chart"))\n)\nres_vis.render_notebook()"}'}, 'id': '', 'type': 'function'}]}))], finish_reason=None), usage=GenerationUsage(input_tokens=415, output_tokens=95))

### classwork2

1, 利用上面的函数，完成可视化问答

2，在streamlit上实现数据统计+可视化问答应用